# Imports

In [1]:
import requests as rq
import pandas as pd
import os
import numpy as np
import json
from dotenv import load_dotenv
from sqlalchemy import create_engine
import copy

# Definitions

In [2]:
base_urls = {
    "films": "https://swapi.dev/api/films/",
    "people": "https://swapi.dev/api/people/",
    "planets": "https://swapi.dev/api/planets/",
    "species": "https://swapi.dev/api/species/",
    "starships": "https://swapi.dev/api/starships/",
    "vehicles": "https://swapi.dev/api/vehicles/"
}

categories = list(base_urls.keys())
categories

['films', 'people', 'planets', 'species', 'starships', 'vehicles']

Each category has different fields that contain information in the form of an url. I will extract the page id from those fields for each category. 

In [3]:
fields = {
    'films' : ["characters", "planets", "starships", "vehicles", "species"],
    'people' : ["homeworld", "films", "species", "vehicles", "starships"],
    'planets' : ['residents', 'films'],
    'species' : ['people', 'films', 'homeworld'],
    'vehicles' : ['pilots', 'films'],
    'starships' : ['pilots', 'films']
    }

# Consume the API

In [4]:
def scrape_category(url):

    # skip the next url for the first page of pager
    next = url    
    items_list = []

    while next:
        # get the content of the url
        response = rq.get(next)

        # success
        if response.status_code == 200:
            content = response.json()
        elif response.status_code == 404:
            print(f'{url} not found!')
            return
        
        next = content['next']
        items = content['results']

        for item in items:
                    
            # remove created and edited fields
            try:
                del(item['created'])
                del(item['edited'])
            except:
                pass
            
            items_list.append(item)            

    return items_list

## Scrape all the categories and store in *starwars_raw.json*
(it takes 12.9 seconds)

In [5]:
if not os.path.exists('./data/starwars_raw.json'):
    raw_dict = {cat : scrape_category(base_urls[cat]) for cat in categories}
    
    # store into a json file
    with open('./data/starwars_raw.json', 'w') as file:
        json.dump(raw_dict, file, indent=4 )
    print('Content from Star Wars API stored in a json file!')

else:
    print('The content already exists in a json file!')
    with open('./data/starwars_raw.json', 'r') as file:
        raw_dict = json.load(file)

The content already exists in a json file!


Function to process the information of an item from a category.
Ex. one character, one planet or one film.

In [6]:
def process_item(item, fields):

    # create a copy of the item dictionary
    item = copy.deepcopy(item)

    # parse the links from starships, vehicles and species
    for field in fields:
        id_values = []
                        
        if item[field]:  # if the field is not empty
            
            # parse the homeworld (just a single string value)
            if field == 'homeworld':
                # get the homeworld id
                item['homeworld'] = int(item['homeworld'].split('/')[-2])

            else:
                for link in item[field]:

                    # parse the id value in the link    
                    id_values.append(int(link.split('/')[-2]))

                # add the id values into the corresponding field key
                # convert list into tuple, as tuples are hashable
                item[field] = tuple(id_values)

                # each character belongs to only 1 species
                # that way, the species contains
        
        # if species is an empty list, it's a human character
        elif field == 'species' and not item['species']:
            item[field] = 1
                    
        # field has no values (empty list)
        else:
            item[field] = ()
    
    # add the id, extracted from the url
    item['id'] = item['url'].split('/')[-2]
    
    # remove created and edited fields
    try:
        del(item['created'])
        del(item['edited'])
    except:
        pass

    return item

# Store the processed data

In [7]:
if not os.path.exists('./data/starwars_processed_items.json'):
    # dictionary to store the processed categories
    processed_dict = {}
    
    # process each item for all the categories
    for k,v in raw_dict.items():
        items_processed = []
        for item in v:
            try:
                items_processed.append(process_item(item, fields[k]))
                processed_dict[k] = items_processed
            except:
                print(f'Error in {k}')
    
    # store the information in a json file
    with open('./data/starwars_processed_items.json', 'w') as file:
        json.dump(processed_dict, file, indent = 4)

# the file already exists, so load it
else:
    with open('./data/starwars_processed_items.json', 'r') as file:
        processed_dict= json.load(file)

        # convert lists into tuples after reading from json file
        for cat in categories:
            for item in processed_dict[cat]:
                for field in fields[cat]:
                    try:
                        item[field] = tuple(item[field])
                    # the content of the field is an integer
                    # and not a list. Cannot create a tuple from
                    # an integer using tuple(int)
                    except:
                        pass  # do not convert into a tuple, leave it as integer

    print('Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.')

Processed data already existed, so the *categories_dict_processed* dictionary will be created from json file.


# Dataframes

## Create the dataframes
Create a dictionary to store the dataframes from each category

In [8]:
dataframes = dict.fromkeys(categories)

In [9]:
for cat in categories:
    df = pd.DataFrame(processed_dict[cat])
    #df['id'] = df.index + 1

    # rename columns to add '_id' to the "fields"
    rename_dict = {field : f'{field}_id' for field in fields[cat]}
    rename_dict.update({'id' : f'{cat}_id'})
    df.rename(columns = rename_dict, inplace = True)

    # reorder the columns to place id in first place
    all_columns_but_cat_id = [col for col in df.columns if col != f'{cat}_id']
    sorted_columns = [f'{cat}_id'] + all_columns_but_cat_id
    dataframes[cat] = df[sorted_columns]
#df.to_csv('./data/starwars_characters.csv', index = False)

## Rename some columns
Make some renamings to the column names

In [10]:
col_rename_dict= {
    
    'films': {
        'characters_id': 'character_id',
        'films_id': 'film_id',
        'planets_id': 'planet_id',
        'episode_id': 'episode',
        'starships_id': 'starship_id',
        'vehicles_id': 'vehicle_id',
    },
    
    'people': {
        'people_id': 'character_id',
        'films_id': 'film_id',
        'vehicles_id': 'vehicle_id',
        'starships_id': 'starship_id'
    },

    'planets': {
        'planets_id': 'planet_id',
        'films_id': 'film_id'
    },

    'species': {
        'people_id': 'character_id',
        'films_id': 'film_id'
    },

    'vehicles': {
        'pilots_id': 'pilot_id',
        'vehicles_id': 'vehicle_id',
        'films_id': 'film_id'
    },

    'starships': {
        'pilots_id': 'pilot_id',
        'starships_id': 'starship_id',
        'films_id': 'film_id'
    }
}

In [11]:
for cat in categories:
    dataframes[cat].rename(columns=col_rename_dict[cat], inplace=True)

## Clean the datasets

### Clean people

In [12]:
df = dataframes['people']
df.sample(5)

,character_id,name,height,mass,hair_color,skin_color,eye_color,birth_year,gender,homeworld_id,film_id,species_id,vehicle_id,starship_id,url
6,7,Beru Whitesun lars,165,75,brown,light,blue,47BBY,female,1,"(1, 5, 6)",1,(),(),https://swapi.dev/api/people/7/
27,29,Arvel Crynyd,unknown,unknown,brown,fair,brown,unknown,male,28,"(3,)",1,(),"(28,)",https://swapi.dev/api/people/29/
39,41,Sebulba,112,40,none,"grey, red",orange,unknown,male,35,"(4,)","(14,)",(),(),https://swapi.dev/api/people/41/
19,21,Palpatine,170,75,grey,pale,yellow,82BBY,male,8,"(2, 3, 4, 5, 6)",1,(),(),https://swapi.dev/api/people/21/
80,82,Sly Moore,178,48,none,pale,white,unknown,female,60,"(5, 6)",1,(),(),https://swapi.dev/api/people/82/


In [13]:
df.dtypes

character_id    object
name            object
height          object
mass            object
hair_color      object
skin_color      object
eye_color       object
birth_year      object
gender          object
homeworld_id     int64
film_id         object
species_id      object
vehicle_id      object
starship_id     object
url             object
dtype: object

In [14]:
df.mass = df.mass.replace('unknown', np.nan)
df.mass = df.mass.str.replace(',', '', regex=False)
df.mass = df.mass.astype('float') 

In [15]:
df.height = df.height.replace('unknown', np.nan).astype('float')

In [16]:
df.hair_color = df.hair_color.replace('n/a', np.nan)

In [17]:
df.birth_year = df.birth_year.str.replace('BBY', ' BBY')

### Clean films

In [18]:
df = dataframes['films']
df.sample(5)

,film_id,title,episode,opening_crawl,director,producer,release_date,character_id,planet_id,starship_id,vehicle_id,species_id,url
3,4,The Phantom Menace,1,Turmoil has engulfed the\r\nGalactic Republic....,George Lucas,Rick McCallum,1999-05-19,"(2, 3, 10, 11, 16, 20, 21, 32, 33, 34, 35, 36,...","(1, 8, 9)","(31, 32, 39, 40, 41)","(33, 34, 35, 36, 37, 38, 42)","(1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, ...",https://swapi.dev/api/films/4/
4,5,Attack of the Clones,2,There is unrest in the Galactic\r\nSenate. Sev...,George Lucas,Rick McCallum,2002-05-16,"(2, 3, 6, 7, 10, 11, 20, 21, 22, 33, 35, 36, 4...","(1, 8, 9, 10, 11)","(21, 32, 39, 43, 47, 48, 49, 52, 58)","(4, 44, 45, 46, 50, 51, 53, 54, 55, 56, 57)","(1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, ...",https://swapi.dev/api/films/5/
2,3,Return of the Jedi,6,Luke Skywalker has returned to\r\nhis home pla...,Richard Marquand,"Howard G. Kazanjian, George Lucas, Rick McCallum",1983-05-25,"(1, 2, 3, 4, 5, 10, 13, 14, 16, 18, 20, 21, 22...","(1, 5, 7, 8, 9)","(2, 3, 10, 11, 12, 15, 17, 22, 23, 27, 28, 29)","(8, 16, 18, 19, 24, 25, 26, 30)","(1, 2, 3, 5, 6, 8, 9, 10, 15)",https://swapi.dev/api/films/3/
5,6,Revenge of the Sith,3,War! The Republic is crumbling\r\nunder attack...,George Lucas,Rick McCallum,2005-05-19,"(1, 2, 3, 4, 5, 6, 7, 10, 11, 12, 13, 20, 21, ...","(1, 2, 5, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19)","(2, 32, 48, 59, 61, 63, 64, 65, 66, 68, 74, 75)","(33, 50, 53, 56, 60, 62, 67, 69, 70, 71, 72, 7...","(1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 2...",https://swapi.dev/api/films/6/
1,2,The Empire Strikes Back,5,It is a dark time for the\r\nRebellion. Althou...,Irvin Kershner,"Gary Kurtz, Rick McCallum",1980-05-17,"(1, 2, 3, 4, 5, 10, 13, 14, 18, 20, 21, 22, 23...","(4, 5, 6, 27)","(3, 10, 11, 12, 15, 17, 21, 22, 23)","(8, 14, 16, 18, 19, 20)","(1, 2, 3, 6, 7)",https://swapi.dev/api/films/2/


In [19]:
df.species_id.unique()

array([(1, 2, 3, 4, 5), (1, 2, 3, 6, 7), (1, 2, 3, 5, 6, 8, 9, 10, 15),
       (1, 2, 6, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27),
       (1, 2, 6, 12, 13, 15, 28, 29, 30, 31, 32, 33, 34, 35),
       (1, 2, 3, 6, 15, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37)],
      dtype=object)

In [20]:
df.dtypes

film_id          object
title            object
episode           int64
opening_crawl    object
director         object
producer         object
release_date     object
character_id     object
planet_id        object
starship_id      object
vehicle_id       object
species_id       object
url              object
dtype: object

In [21]:
df.release_date = pd.to_datetime(df.release_date)

### Clean planets

In [22]:
df = dataframes['planets']
df.sample(5)

,planet_id,name,rotation_period,orbital_period,diameter,climate,gravity,terrain,surface_water,population,residents_id,film_id,url
11,12,Utapau,27,351,12900,"temperate, arid, windy",1 standard,"scrublands, savanna, canyons, sinkholes",0.9,95000000,"(83,)","(6,)",https://swapi.dev/api/planets/12/
37,38,Aleen Minor,unknown,unknown,unknown,unknown,unknown,unknown,unknown,unknown,"(47,)",(),https://swapi.dev/api/planets/38/
38,39,Vulpter,22,391,14900,"temperate, artic",1,"urban, barren",unknown,421000000,"(48,)",(),https://swapi.dev/api/planets/39/
7,8,Naboo,26,312,12120,temperate,1 standard,"grassy hills, swamps, forests, mountains",12,4500000000,"(3, 21, 35, 36, 37, 38, 39, 42, 60, 61, 66)","(3, 4, 5, 6)",https://swapi.dev/api/planets/8/
46,47,Iktotch,22,481,unknown,"arid, rocky, windy",1,rocky,unknown,unknown,"(56,)",(),https://swapi.dev/api/planets/47/


In [23]:
df.dtypes

planet_id          object
name               object
rotation_period    object
orbital_period     object
diameter           object
climate            object
gravity            object
terrain            object
surface_water      object
population         object
residents_id       object
film_id            object
url                object
dtype: object

In [24]:
for col in ['rotation_period', 'orbital_period', 'diameter', 'surface_water', 'population']:
    df[col] = df[col].replace('unknown', np.nan).astype('float')

In [25]:
df.population = df.population / 1E6
df.rename(columns = {'population' : 'population_millions'}, inplace = True)

In [26]:
df.gravity.unique()

array(['1 standard', '1.1 standard', 'N/A',
       '1.5 (surface), 1 standard (Cloud City)', '0.85 standard',
       '0.9 standard', '0.56 standard', '0.75 standard', 'unknown',
       '0.62 standard', '1', '1.56', '0.9', '0.98'], dtype=object)

In [27]:
df.gravity = df.gravity.str.replace(' standard', '').str.replace(df.gravity[5], '1.5')
df.gravity = df.gravity.replace('unknown', np.nan)

### Clean species

In [28]:
df = dataframes['species']
df.sample(5)

,species_id,name,classification,designation,average_height,skin_colors,hair_colors,eye_colors,average_lifespan,homeworld_id,language,character_id,film_id,url
20,21,Nautolan,amphibian,sentient,180,"green, blue, brown, red",none,black,70,44,Nautila,"(53,)","(4,)",https://swapi.dev/api/species/21/
33,34,Muun,mammal,sentient,190,"grey, white",none,black,100,57,Muun,"(77,)","(5, 6)",https://swapi.dev/api/species/34/
9,10,Sullustan,mammal,sentient,180,pale,none,black,unknown,33,Sullutese,"(31,)","(3,)",https://swapi.dev/api/species/10/
19,20,Cerean,mammal,sentient,200,pale pink,"red, blond, black, white",hazel,unknown,43,Cerean,"(52,)","(4, 6)",https://swapi.dev/api/species/20/
26,27,Chagrian,amphibian,sentient,190,blue,none,blue,unknown,50,Chagria,"(59,)","(4, 6)",https://swapi.dev/api/species/27/


In [29]:
df.dtypes

species_id          object
name                object
classification      object
designation         object
average_height      object
skin_colors         object
hair_colors         object
eye_colors          object
average_lifespan    object
homeworld_id        object
language            object
character_id        object
film_id             object
url                 object
dtype: object

In [30]:
df.average_height = df.average_height.replace('unknown', np.nan).replace('n/a', np.nan).astype('float')

In [31]:
df.average_lifespan = df.average_lifespan.replace('unknown', np.nan).replace('indefinite', 9999).astype('float')

In [32]:
df.loc[1, 'homeworld_id'] = np.nan
df.homeworld_id = df.homeworld_id.astype('float')

### Clean vehicles

In [33]:
df = dataframes['vehicles']
df.sample(5)

,vehicle_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,pilot_id,film_id,url
26,54,SPHA,Self-Propelled Heavy Artillery,Rothana Heavy Engineering,unknown,140,35,25,30,500,7 days,walker,(),"(5,)",https://swapi.dev/api/vehicles/54/
37,73,Droid gunship,HMP droid gunship,"Baktoid Fleet Ordnance, Haor Chall Engineering",60000,12.3,820,0,0,0,none,airspeeder,(),"(6,)",https://swapi.dev/api/vehicles/73/
18,38,Tribubble bongo,Tribubble bongo,Otoh Gunga Bongameken Cooperative,unknown,15,85,1,2,1600,unknown,submarine,"(10, 32)","(4,)",https://swapi.dev/api/vehicles/38/
15,35,Armored Assault Tank,Armoured Assault Tank,Baktoid Armor Workshop,unknown,9.75,55,4,6,unknown,unknown,repulsorcraft,(),"(4,)",https://swapi.dev/api/vehicles/35/
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500,10.4,1200,1,1,50,0,repulsorcraft,(),"(1,)",https://swapi.dev/api/vehicles/6/


In [34]:
df.cost_in_credits = df.cost_in_credits.replace('unknown', np.nan)
df.cost_in_credits = df.cost_in_credits.astype('float')

In [35]:
for col in ['max_atmosphering_speed' ,'crew', 'passengers', 'cargo_capacity']:
    df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan)
    df[col] = df[col].astype('float')

In [36]:
df.length = df.length.replace('unknown', np.nan)
df.length = df.length.astype('float')

In [37]:
df.consumables = df.consumables.replace('0', 'none')

### Clean starships

In [38]:
df = dataframes['starships']
df.sample(5)

,starship_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,hyperdrive_rating,MGLT,starship_class,pilot_id,film_id,url
14,28,A-wing,RZ-1 A-wing Interceptor,"Alliance Underground Engineering, Incom Corpor...",175000,9.6,1300,1,0,40,1 week,1.0,120,Starfighter,"(29,)","(3,)",https://swapi.dev/api/starships/28/
22,47,AA-9 Coruscant freighter,Botajef AA-9 Freighter-Liner,Botajef Shipyards,unknown,390,unknown,unknown,30000,unknown,unknown,unknown,unknown,freighter,(),"(5,)",https://swapi.dev/api/starships/47/
30,64,Naboo star skiff,J-type star skiff,Theed Palace Space Vessel Engineering Corps/Nu...,unknown,29.2,1050,3,3,unknown,unknown,0.5,unknown,yacht,"(10, 35)","(6,)",https://swapi.dev/api/starships/64/
1,3,Star Destroyer,Imperial I-class Star Destroyer,Kuat Drive Yards,150000000,"1,600",975,"47,060",n/a,36000000,2 years,2.0,60,Star Destroyer,(),"(1, 2, 3)",https://swapi.dev/api/starships/3/
28,61,Theta-class T-2c shuttle,Theta-class T-2c shuttle,Cygnus Spaceworks,1000000,18.5,2000,5,16,50000,56 days,1.0,unknown,transport,(),"(6,)",https://swapi.dev/api/starships/61/


In [39]:
df.loc[0, 'crew'] = 165

In [40]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].replace('unknown', np.nan).replace('none', np.nan).replace('n/a', np.nan)
        df[col] = df[col].str.replace(',', '', regex = False).str.replace('km', '')
        #df[col] = df[col].astype('float')
    except Exception as e:
        print(f'Error in {col}: {e}')

In [41]:
for col in ['cost_in_credits', 'length', 'max_atmosphering_speed', 'crew', 'passengers', 'cargo_capacity', 'hyperdrive_rating', 'MGLT']:
    try:
        df[col] = df[col].astype('float')
    except:
        print(f'error with {col}')

## Export clean datasets into csv files

In [42]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv/', exist_ok=True)
        df = dataframes[cat]
        df.to_csv(f'{data_path}/csv/{cat}_dataframe.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv/ as csv files!')

File films_dataframe.csv already exist!
File people_dataframe.csv already exist!
File planets_dataframe.csv already exist!
File species_dataframe.csv already exist!
File starships_dataframe.csv already exist!
File vehicles_dataframe.csv already exist!
Dataframes of each normalized category are stored in ./data/csv/ as csv files!


# Junction tables

(many-to-many relationships in the database)

1. **films_people**: Links films to the characters that appeared in them.

    - character_id: Foreign Key referencing the `people` table.
    - film_id: Foreign Key referencing the films table.

2. **films_planets**: Links films to the planets that appeared in them.

    - planet_id: Foreign Key referencing the `planets` table.
    - film_id: Foreign Key referencing the `films` table.

3. **films_starships**: Links films to the starships that appeared in them.

    - starship_id: Foreign Key referencing the `starships` table.
    - film_id: Foreign Key referencing the `films` table.

4. **films_vehicles**: Links films to the vehicles that appeared in them.

    - vehicle_id: Foreign Key referencing the `vehicles` table.
    - film_id: Foreign Key referencing the `films` table.

5. **films_species**: Links films to the species that appeared in them.

    - species_id: Foreign Key referencing the `species` table.
    - film_id: Foreign Key referencing the `films` table.

6. **people_starships**: Links people (pilots) to the starships they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - starship_id: Foreign Key referencing the `starships` table.

7. **people_vehicles**: Links people (pilots) to the vehicles they have piloted.

    - character_id: Foreign Key referencing the `people` table.
    - vehicle_id: Foreign Key referencing the vehicles table.

In [43]:
junction_tables = [
    'people_vehicles', 
    'people_starships',
    
    'films_people',
    'films_planets',
    'films_starships',
    'films_vehicles',
    'films_species'
    ]

junction_tables_dict = {i:None for i in junction_tables}

## Junction tables for people:

In [44]:
columns = ['vehicle_id', 'starship_id']
data = dataframes['people'].loc[:, columns + ['character_id']]

for col in columns:
    table_name = f'people_{col}'.replace('_id', 's')
    junction_tables_dict[table_name] = (data.loc[:, ['character_id', col]]
                                        .explode(col)
                                        .explode('character_id')
                                        .dropna()
                                        .reset_index(drop = True)
                                        .sort_values('character_id')
                                        )

## Junction tables for films:

In [45]:
columns = ['character_id', 'species_id', 'planet_id', 'vehicle_id', 'starship_id']
data = dataframes['films'].loc[:, columns + ['film_id']]

for col in columns:
    table_name = f'films_{col}'
    if col == 'species_id':
        table_name = table_name.replace('_id', '')
    elif col == 'character_id':
        table_name = 'films_people'
    else:
        table_name = table_name.replace('_id', 's')
        
    junction_tables_dict[table_name] = (data.loc[:, ['film_id', col ]]
                                        .explode(col)
                                        .reset_index(drop = True)
                                        .sort_values('film_id')
    )

# Normalization
Next step is to normalize the datasets in order to create the database.

In [46]:
dataframes_normalized = copy.deepcopy(dataframes)

In [47]:
columns_to_drop = {
    'films' : ['character_id', 'planet_id', 'species_id', 'vehicle_id', 'starship_id'],
    'people' : ['film_id', 'species_id', 'vehicle_id', 'starship_id'],
    'planets' : ['residents_id', 'film_id'],
    'species' : ['character_id', 'film_id'],
    'starships' : ['pilot_id', 'film_id'],
    'vehicles' : ['pilot_id', 'film_id'],
}  

### Drop the corresponding columns in order to normalize the tables

In [48]:
for cat in dataframes_normalized.keys():
    dataframes_normalized[cat].drop(columns_to_drop[cat], axis='columns', inplace = True)

## Store the normalized dataframes

In [49]:
data_path = './data'
for cat in categories:
    filename = f'{cat}_dataframe.csv'
    if os.path.exists(f'{data_path}/csv_normalized/{filename}'):
        print(f'File {filename} already exist!')
        pass
    else:
        os.makedirs(f'{data_path}/csv_normalized/', exist_ok=True)
        df = dataframes_normalized[cat]
        df.to_csv(f'{data_path}/csv_normalized/{cat}_dataframe_normalized.csv', index = False)
print(f'Dataframes of each normalized category are stored in {data_path}/csv_normalized/ as csv files!')

Dataframes of each normalized category are stored in ./data/csv_normalized/ as csv files!


In [50]:
dataframes_normalized['vehicles']

,vehicle_id,name,model,manufacturer,cost_in_credits,length,max_atmosphering_speed,crew,passengers,cargo_capacity,consumables,vehicle_class,url
0,4,Sand Crawler,Digger Crawler,Corellia Mining Corporation,150000.0,36.80,30.0,46.0,30.0,50000.0,2 months,wheeled,https://swapi.dev/api/vehicles/4/
1,6,T-16 skyhopper,T-16 skyhopper,Incom Corporation,14500.0,10.40,1200.0,1.0,1.0,50.0,none,repulsorcraft,https://swapi.dev/api/vehicles/6/
2,7,X-34 landspeeder,X-34 landspeeder,SoroSuub Corporation,10550.0,3.40,250.0,1.0,1.0,5.0,unknown,repulsorcraft,https://swapi.dev/api/vehicles/7/
3,8,TIE/LN starfighter,Twin Ion Engine/Ln Starfighter,Sienar Fleet Systems,NaN,6.40,1200.0,1.0,0.0,65.0,2 days,starfighter,https://swapi.dev/api/vehicles/8/
4,14,Snowspeeder,t-47 airspeeder,Incom corporation,NaN,4.50,650.0,2.0,0.0,10.0,none,airspeeder,https://swapi.dev/api/vehicles/14/
5,16,TIE bomber,TIE/sa bomber,Sienar Fleet Systems,NaN,7.80,850.0,1.0,0.0,NaN,2 days,space/planetary bomber,https://swapi.dev/api/vehicles/16/
6,18,AT-AT,All Terrain Armored Transport,"Kuat Drive Yards, Imperial Department of Milit...",NaN,20.00,60.0,5.0,40.0,1000.0,unknown,assault walker,https://swapi.dev/api/vehicles/18/
7,19,AT-ST,All Terrain Scout Transport,"Kuat Drive Yards, Imperial Department of Milit...",NaN,2.00,90.0,2.0,0.0,200.0,none,walker,https://swapi.dev/api/vehicles/19/
8,20,Storm IV Twin-Pod cloud car,Storm IV Twin-Pod,Bespin Motors,75000.0,7.00,1500.0,2.0,0.0,10.0,1 day,repulsorcraft,https://swapi.dev/api/vehicles/20/
9,24,Sail barge,Modified Luxury Sail Barge,Ubrikkian Industries Custom Vehicle Division,285000.0,30.00,100.0,26.0,500.0,2000000.0,Live food tanks,sail barge,https://swapi.dev/api/vehicles/24/


# Insert data into the database

## Load database parameters from `.env` file

In [51]:
load_dotenv()

DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

## Create the db connection

In [52]:
connection_string = (
    f'mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}'
)

# --- 4. Create the SQLAlchemy Engine ---
try:
    engine = create_engine(connection_string)
    print("SQLAlchemy Engine created successfully. 🛠️")
except Exception as e:
    print(f"Error creating engine: {e}")

SQLAlchemy Engine created successfully. 🛠️


## Populate the data into the database

The order of tables to be filled must be:
1. planets
2. species
3. starships
4. vehicles
5. films
6. people

In [53]:
categories_sorted = ['planets', 'species', 'vehicles', 'starships', 'films', 'people']

### Insert the category tables into the database

In [54]:
def insert_category(cat, dictionary):
    df = dictionary[cat]
    try:
        df.to_sql(name=cat, con=engine, if_exists='append', index=False)
        print(f"DataFrame for category '{cat}' inserted successfully into the database. ✅\n")
    except Exception as e:
        print(f"\\ Error inserting DataFrame for category '{cat}': \n{e}\n\n")

In [55]:
for cat in categories_sorted:
    query = f'select * from {cat} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(cat, dataframes_normalized)
    else:
        print(f'{cat} table already exists in database!')

DataFrame for category 'planets' inserted successfully into the database. ✅

DataFrame for category 'species' inserted successfully into the database. ✅

DataFrame for category 'vehicles' inserted successfully into the database. ✅

DataFrame for category 'starships' inserted successfully into the database. ✅

DataFrame for category 'films' inserted successfully into the database. ✅

DataFrame for category 'people' inserted successfully into the database. ✅



### Insert junction tables

In [57]:
for table in junction_tables_dict.keys():
    query = f'select * from {table} limit 1 ;'
    # if table is empty, fill it with the corresponding data
    if pd.read_sql(query, con = engine).shape[0] == 0:
        insert_category(table, junction_tables_dict)
    else:
        print(f'{cat} table already exists in database!') 

DataFrame for category 'people_vehicles' inserted successfully into the database. ✅

DataFrame for category 'people_starships' inserted successfully into the database. ✅

DataFrame for category 'films_people' inserted successfully into the database. ✅

DataFrame for category 'films_planets' inserted successfully into the database. ✅

DataFrame for category 'films_starships' inserted successfully into the database. ✅

DataFrame for category 'films_vehicles' inserted successfully into the database. ✅

DataFrame for category 'films_species' inserted successfully into the database. ✅

